Boyd, John H., et al. “The Stock Market's Reaction to Unemployment News: Why Bad News Is Usually Good for Stocks.” The Journal of Finance, vol. 60, no. 2, 2005, pp. 649–672. JSTOR, www.jstor.org/stable/3694763


In [2]:
import matplotlib.pyplot as plt
from odo import odo
import pandas as pd
import blaze as bz
import numpy as np
import scipy.stats as stats
import datetime
import statsmodels.tsa as tsa
from statsmodels import regression
import statsmodels.api as sm
from quantopian.interactive.data.quandl import fred_unrate, fred_gdp

In [19]:
fred_unrate.sort('asof_date',ascending=True)

,value,asof_date,timestamp
0,3.4,1948-01-01,1948-01-02
1,3.8,1948-02-01,1948-02-02
2,4.0,1948-03-01,1948-03-02
3,3.9,1948-04-01,1948-04-02
4,3.5,1948-05-01,1948-05-02
5,3.6,1948-06-01,1948-06-02
6,3.6,1948-07-01,1948-07-02
7,3.9,1948-08-01,1948-08-02
8,3.8,1948-09-01,1948-09-02
9,3.7,1948-10-01,1948-10-02


In [16]:
unemployment = bz.compute(fred_unrate).set_index(['asof_date']).sort_index()#.drop('timestamp', 1)
# data.columns = ['unrate']
# data['GDP'] = bz.compute(fred_gdp).set_index(['asof_date']).sort_index().drop('timestamp', 1)
# data['unrate_change'] = data['unrate'].pct_change(1)
# data['GDP_change'] = data['GDP'].pct_change(1)

In [17]:
dates = pd.to_datetime(data.index, yearfirst = True)

new_index = []

for date in dates:
    next_month = datetime.datetime(date.year + (date.month / 12),
                                  (date.month % 12) + 1, 1)
    for i in range(7):
        test_date = datetime.datetime(next_month.year, next_month.month, i+1)
        if (test_date.weekday() == 5):
            annc_date = test_date
            
    new_index.append(annc_date)
    
unemployment['new_dates'] = new_index

# data = data.set_index(['fixed_dates'])

In [18]:
unemployment.loc['2015-05-01':'2016-03-01']

,value,timestamp,new_dates
asof_date,,,
2015-05-01,5.5,2015-05-02 00:00:00.000000,2015-06-06
2015-06-01,5.3,2015-06-02 00:00:00.000000,2015-07-04
2015-07-01,5.3,2015-07-02 00:00:00.000000,2015-08-01
2015-08-01,5.1,2015-08-02 00:00:00.000000,2015-09-05
2015-09-01,5.1,2015-09-02 00:00:00.000000,2015-10-03
2015-10-01,5.0,2015-10-02 00:00:00.000000,2015-11-07
2015-11-01,5.0,2015-11-02 00:00:00.000000,2015-12-05
2015-12-01,5.0,2016-01-09 04:08:01.432194,2016-01-02
2016-01-01,4.9,2016-02-06 04:02:32.460830,2016-02-06


In [166]:
# Getting an understanding of the size and structure of the data
print "-------- FRED Unemployment Rate Data --------"
print "%-12s %-15s %-13s %s" % ('Start Date:', data['unrate'].index[0].date(), 'End Date:', data['unrate'].index[-1].date())
print "%-12s %-15s %-13s %s" % ('Min Value:', data['unrate'].min(), 'Max Value:', data['unrate'].max())
print "%-12s %-15s %-13s %s" % ('Avg Value:', data['unrate'].mean(), 'Median Value:', data['unrate'].median())
print "Frequency: monthly\n"

print "-------- FRED GDP Data --------"
print "%-12s %-15s %-13s %s" % ('Start Date:', data['GDP'].index[0].date(), 'End Date:', data['GDP'].index[-1].date())
print "%-12s %-15s %-13s %s" % ('Min Value:', data['GDP'].min(), 'Max Value:', data['GDP'].max())
print "%-12s %-15s %-13s %s" % ('Avg Value:', data['GDP'].mean(), 'Median Value:', data['GDP'].median())
print "Frequency: quarterly"

# Fill monthly data down through days
data = data.ffill()

# Add boolean column denoting a contraction(true)/expansion(false)
data['contraction'] = data['GDP_change'].map(lambda x: x<0).shift(3)

data

-------- FRED Unemployment Rate Data --------
Start Date:  1948-02-06      End Date:     2017-06-02
Min Value:   2.5             Max Value:    10.8
Avg Value:   5.80300120048   Median Value: 5.6
Frequency: monthly

-------- FRED GDP Data --------
Start Date:  1948-02-06      End Date:     2017-06-02
Min Value:   266.2           Max Value:    19007.3
Avg Value:   5706.81732852   Median Value: 3367.1
Frequency: quarterly


,unrate,GDP,unrate_change,GDP_change,fixed dates,contraction
fixed_dates,,,,,,
1948-02-06,3.4,266.2,NaN,NaN,1948-02-06,NaN
1948-03-05,3.8,266.2,0.117647,NaN,1948-03-05,NaN
1948-04-02,4.0,266.2,0.052632,NaN,1948-04-02,NaN
1948-05-07,3.9,272.9,-0.025000,0.025169,1948-05-07,False
1948-06-04,3.5,272.9,-0.102564,0.025169,1948-06-04,False
1948-07-02,3.6,272.9,0.028571,0.025169,1948-07-02,False
1948-08-06,3.6,279.5,0.000000,0.024185,1948-08-06,False
1948-09-03,3.9,279.5,0.083333,0.024185,1948-09-03,False
1948-10-01,3.8,279.5,-0.025641,0.024185,1948-10-01,False


In [167]:
# Fill contraction column with NBER data (www.nber.org/cycles.html)

peaks = [
    '01-01-1980',
    '07-01-1981',
    '07-01-1990',
    '03-01-2001',
    '12-01-2007']

troughs = [
    '07-01-1980',
    '11-01-1982',
    '03-01-1991',
    '11-01-2001',
    '06-01-2009']

data['contraction'].loc[peaks[0]:troughs[4]] = False

for i in range(len(peaks)):
    data['contraction'].loc[peaks[i]:troughs[i]] = True

# Testing w Spy

In [487]:
benchmark = get_pricing('SPY',
                        fields = 'price',
                        start_date = '2002-01-01',
                        end_date = '2017-01-01')

In [511]:
# Store returns for day after unemployment news
post_strike_returns = []

for date in data['2002-01-01':'2016-12-30'].index:
    if (data.loc[date]['contraction'] & (data.loc[date]['unrate_change'] > 0)):
        price_spot = benchmark.iloc[benchmark.index.get_loc(date,method='nearest')]
        price_next = benchmark.iloc[benchmark.index.get_loc(date,method='nearest')+1]
        post_strike_returns.append((price_next - price_spot)/price_spot)

print 'Mean returns after unrate up & expansion:', np.mean(post_strike_returns)
print 'Mean return:',np.mean(benchmark.pct_change(1))
post_strike_returns

Mean returns after unrate up & expansion: 0.00803704867715
Mean return: 0.000319791691461


[0.0028309883031021272,
 -0.0056189927116586927,
 -0.0018284301981009044,
 0.014776646936081377,
 -0.007657171331284753,
 0.029376274887235349,
 0.03493548130680163,
 0.00095801911125290159,
 0.015787131379851486,
 -0.0031894609117697607]

In [461]:
benchmark = get_pricing('SPY',
                        fields = 'price',
                        start_date = '2002-01-01',
                        end_date = '2017-01-01')

In [462]:
post_strike_returns = []
for date in data['2002-01-01':'2017-01-01'].index:
    if (data.loc[date]['contraction'] & (data.loc[date]['unrate_change'] > 0)):
        price_spot = benchmark.iloc[benchmark.index.get_loc(date,method='nearest')]
        price_next = benchmark.iloc[benchmark.index.get_loc(date,method='nearest')+1]
        post_strike_returns.append((price_next - price_spot)/price_spot)

print 'Mean returns after unrate rises & expansion:', np.mean(post_strike_returns)
print 'Mean return:',np.mean(benchmark.pct_change(1))

Mean returns after unrate up & expansion: 0.00386474448032
Mean return: 0.000319791691461


# Trying to replicate the paper's unemployment model 

We need the historical model predictions to determine whether the unemployment news was a 'surprise' or not.

In [170]:
ind_pro = local_csv('industrial_prod_rate.csv').set_index([pd.date_range(start='1919-01-01', end = '2017-05-01', freq = 'MS')])['1948-01-02':'2017-06-02'].drop('DATE', 1).astype(float)
data['ind_pro']= data['unrate']
data['ind_pro'][1:]=ind_pro['INDPRO'].astype(float)
data['ind_pro_change']=data['ind_pro'].pct_change(1)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [171]:
data

,unrate,GDP,unrate_change,GDP_change,fixed dates,contraction,ind_pro,ind_pro_change
fixed_dates,,,,,,,,
1948-02-06,3.4,266.2,NaN,NaN,1948-02-06,NaN,3.4000,NaN
1948-03-05,3.8,266.2,0.117647,NaN,1948-03-05,NaN,14.8535,3.368676
1948-04-02,4.0,266.2,0.052632,NaN,1948-04-02,NaN,14.6866,-0.011236
1948-05-07,3.9,272.9,-0.025000,0.025169,1948-05-07,False,14.7144,0.001893
1948-06-04,3.5,272.9,-0.102564,0.025169,1948-06-04,False,14.9647,0.017011
1948-07-02,3.6,272.9,0.028571,0.025169,1948-07-02,False,15.1594,0.013011
1948-08-06,3.6,279.5,0.000000,0.024185,1948-08-06,False,15.1594,0.000000
1948-09-03,3.9,279.5,0.083333,0.024185,1948-09-03,False,15.1038,-0.003668
1948-10-01,3.8,279.5,-0.025641,0.024185,1948-10-01,False,14.9926,-0.007362
